In [1]:
import pandas as pd
import requests
import os
from google.cloud import bigquery

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'secret.json'

bq_client = bigquery.Client()

[parties](https://data.cityofnewyork.us/City-Government/ACRIS-Real-Property-Parties/636b-3b5g)

In [3]:

ACRIS=f'https://data.cityofnewyork.us/resource/8h5j-fqxa.json?$'

In [4]:
acris_count=f'https://data.cityofnewyork.us/resource/8h5j-fqxa.json?$select=count(1) as cnt'
cnt_req=requests.get(acris_count).json()
nrows=int(cnt_req[0]['cnt'])

In [5]:
nrows

20560075

In [6]:
limit=10**6
offset=0

offs=list(range(0, nrows, limit))

reqs=[f'{ACRIS}limit={limit}&$offset={o}&$order=good_through_date DESC' for o in offs]

In [7]:
table_id='pluto-panel.real_estate.raw_acris_legals'

job_config = bigquery.LoadJobConfig(
        autodetect=True,
        write_disposition='WRITE_APPEND'

        )

In [8]:
prog=1
for r in reqs:
    print(f'reading {prog} / {len(reqs)}: {r}')
    d=requests.get(r).json()
    print('making dataframe')
    d2=pd.DataFrame(d)
    print('loading to bigquery')
    job = bq_client.load_table_from_dataframe(
            d2, table_id, job_config=job_config)
    job.result()
    prog+=1
    
    

reading 1 / 21: https://data.cityofnewyork.us/resource/8h5j-fqxa.json?$limit=1000000&$offset=0&$order=good_through_date DESC
making dataframe
loading to bigquery
reading 2 / 21: https://data.cityofnewyork.us/resource/8h5j-fqxa.json?$limit=1000000&$offset=1000000&$order=good_through_date DESC
making dataframe
loading to bigquery
reading 3 / 21: https://data.cityofnewyork.us/resource/8h5j-fqxa.json?$limit=1000000&$offset=2000000&$order=good_through_date DESC
making dataframe
loading to bigquery
reading 4 / 21: https://data.cityofnewyork.us/resource/8h5j-fqxa.json?$limit=1000000&$offset=3000000&$order=good_through_date DESC
making dataframe
loading to bigquery
reading 5 / 21: https://data.cityofnewyork.us/resource/8h5j-fqxa.json?$limit=1000000&$offset=4000000&$order=good_through_date DESC
making dataframe
loading to bigquery
reading 6 / 21: https://data.cityofnewyork.us/resource/8h5j-fqxa.json?$limit=1000000&$offset=5000000&$order=good_through_date DESC
making dataframe
loading to bigquer

In [ ]:
table = bq_client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)